In [1]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so

import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from autogluon.tabular import TabularDataset, TabularPredictor

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|_______30th_Jun_2023_______|



In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
# train
# test
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [4]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [5]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
train['facts'] = df['facts']
df = pd.DataFrame(train['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
train['first_party'] = df['first_party']
df = pd.DataFrame(train['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
train['second_party'] = df['second_party']
train_cleansed = train.drop(columns='ID')
train_cleansed


,first_party,second_party,facts,first_party_winner
0,phil st amant,herman thompson,on june 27 1962 phil st amant cidate for publi...,1
1,stephen duncan,lawrence owens,ramon nelson was riding his bike when he suffe...,0
2,billy joe magwood,tony patterson warden,an alabama state court convicted billy joe mag...,1
3,linkletter,walker,victor linkletter was convicted in state court...,0
4,william earl fikes,alabama,on april 24 1953 in selma alabama an intruder ...,1
...,...,...,...,...
2473,hollyfrontier cheyenne refining llc,renewable fuels association,congress amended the clean air act through the...,1
2474,grupo mexicano de desarrollo,alliance bond fund,alliance bond fund an investment fund purchase...,1
2475,peguero,united states,in 1992 the district court sentenced manuel pe...,0
2476,immigration naturalization service,st cyr,on march 1996 enrico st cyr lawful permanent r...,0


In [6]:
df = pd.DataFrame(test['facts'])
df = dlc.law_preprocessor(df, 'facts')
test['facts'] = df['facts']
df = pd.DataFrame(test['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
test['first_party'] = df['first_party']
df = pd.DataFrame(test['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
test['second_party'] = df['second_party']
test_cleansed = test.drop(columns='ID')
test_cleansed

,first_party,second_party,facts
0,salerno,united states,the 1984 bail reform act allowed the federal c...
1,milberg weiss bershad hynes lerach,lexecon,lexecon was defendant in class action lawsuit ...
2,no 07 582 title federal communications commission,fox television stations,in 2002 2003 fox television stations broadcast...
3,harold kaufman,united states,during his trial for armed robbery of federall...
4,berger,hanlon,in 1993 magistrate judge issued warrant author...
...,...,...,...
1235,haitian centers council,chris sale acting commissioner immigration nat...,according to executive order no 12807 signed b...
1236,whitman,american trucking associations,section 109 of the clean air act requires the ...
1237,linda matteo john madigan,william barr,linda matteo john madigan created plan for uti...
1238,washington state apple advertising commission,hunt,in 1972 the north carolina board of agricultur...


## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [7]:
train_facts = pd.DataFrame(test_cleansed['facts'])
train_facts

,facts
0,the 1984 bail reform act allowed the federal c...
1,lexecon was defendant in class action lawsuit ...
2,in 2002 2003 fox television stations broadcast...
3,during his trial for armed robbery of federall...
4,in 1993 magistrate judge issued warrant author...
...,...
1235,according to executive order no 12807 signed b...
1236,section 109 of the clean air act requires the ...
1237,linda matteo john madigan created plan for uti...
1238,in 1972 the north carolina board of agricultur...


In [8]:
test_fact = pd.DataFrame(test_cleansed['facts'])
test_fact

,facts
0,the 1984 bail reform act allowed the federal c...
1,lexecon was defendant in class action lawsuit ...
2,in 2002 2003 fox television stations broadcast...
3,during his trial for armed robbery of federall...
4,in 1993 magistrate judge issued warrant author...
...,...
1235,according to executive order no 12807 signed b...
1236,section 109 of the clean air act requires the ...
1237,linda matteo john madigan created plan for uti...
1238,in 1972 the north carolina board of agricultur...


model_input_df = dlc.bert_tokenizer(train_facts, 'facts')

model_input_df.info()

df_part_1, df_part_2, df_part_3, df_part_4, df_part_5, df_part_6, df_part_7, df_part_8, df_part_9, df_part_10, df_part_11, df_part_12, df_part_13, df_part_14, df_part_15, df_part_16, df_part_17, df_part_18, df_part_19, df_part_20, df_part_21, df_part_22, df_part_23, df_part_24, df_part_25, df_part_26 = so.df_divider(train_cleansed, 'facts')
df_part_1 = pd.DataFrame(df_part_1)
df_part_2 = pd.DataFrame(df_part_2)
df_part_3 = pd.DataFrame(df_part_3)
df_part_4 = pd.DataFrame(df_part_4)
df_part_5 = pd.DataFrame(df_part_5)
df_part_6 = pd.DataFrame(df_part_6)
df_part_7 = pd.DataFrame(df_part_7)
df_part_8 = pd.DataFrame(df_part_8)
df_part_9 = pd.DataFrame(df_part_9)
df_part_10 = pd.DataFrame(df_part_10)
df_part_11 = pd.DataFrame(df_part_11)
df_part_12 = pd.DataFrame(df_part_12)
df_part_13 = pd.DataFrame(df_part_13)
df_part_14 = pd.DataFrame(df_part_14)
df_part_15 = pd.DataFrame(df_part_15)
df_part_16 = pd.DataFrame(df_part_16)
df_part_17 = pd.DataFrame(df_part_17)
df_part_18 = pd.DataFrame(df_part_18)
df_part_19 = pd.DataFrame(df_part_19)
df_part_20 = pd.DataFrame(df_part_20)
df_part_21 = pd.DataFrame(df_part_21)
df_part_22 = pd.DataFrame(df_part_22)
df_part_23 = pd.DataFrame(df_part_23)
df_part_24 = pd.DataFrame(df_part_24)
df_part_25 = pd.DataFrame(df_part_25)
df_part_26 = pd.DataFrame(df_part_26)
print(df_part_1)

embedded_df_1 = dlc.auto_tokenizer(train_cleansed, 'facts')
embedded_df_1

embedded_df_1 = embedded_df_1.rename(columns={0:'facts_berted'})

embedded_df_1.to_csv('./embeddings/facts_embedded.csv', index=False)

first_party_berted = dlc.auto_tokenizer(train_cleansed, 'first_party')
first_party_berted = first_party_berted.rename(columns={0:'first_party_berted'})
first_party_berted.to_csv('./embeddings/first_party_berted.csv', index=False)

second_party_berted = dlc.auto_tokenizer(train_cleansed, 'second_party')
second_party_berted = second_party_berted.rename(columns={0:'second_party_berted'})
second_party_berted.to_csv('./embeddings/second_party_berted.csv', index=False)

facts_berted = dlc.auto_tokenizer(train_cleansed, 'facts')

facts_berted = facts_berted.rename(columns={0:'facts_berted'})
facts_berted.to_csv('./embeddings/facts_berted.csv', index=False)

In [9]:
first_party_berted = pd.read_csv('./embeddings/first_party_berted.csv')
second_party_berted = pd.read_csv('./embeddings/second_party_berted.csv')
facts_berted = pd.read_csv('./embeddings/facts_berted.csv')

In [10]:
all_ready_to_ml = pd.concat([first_party_berted['first_party_berted'], second_party_berted['second_party_berted'], facts_berted['facts_berted'], train_cleansed['first_party_winner']], axis=1)
all_ready_to_ml.to_csv('./embeddings/1_train_ready_to_ml.csv', index=False)


test_first_party_berted = dlc.auto_tokenizer(test_cleansed, 'first_party')
test_first_party_berted = test_first_party_berted.rename(columns={0:'first_party_berted'})
test_first_party_berted.to_csv('./embeddings/test_first_party_berted.csv', index=False)

test_second_party_berted = dlc.auto_tokenizer(test_cleansed, 'second_party')
test_second_party_berted = test_second_party_berted.rename(columns={0:'second_party_berted'})
test_second_party_berted.to_csv('./embeddings/test_second_party_berted.csv', index=False)

test_facts_berted = dlc.auto_tokenizer(test_cleansed, 'facts')
test_facts_berted = test_facts_berted.rename(columns={0:'facts_berted'})
test_facts_berted.to_csv('./embeddings/test_facts_berted.csv', index=False)

In [11]:
test_first_party_berted = pd.read_csv('./embeddings/test_first_party_berted.csv')
test_second_party_berted = pd.read_csv('./embeddings/test_second_party_berted.csv')
test_facts_berted = pd.read_csv('./embeddings/test_facts_berted.csv')

In [12]:
test_ready_to_ml = pd.concat([test_first_party_berted['first_party_berted'], test_second_party_berted['second_party_berted'], test_facts_berted['facts_berted']], axis=1)
test_ready_to_ml.to_csv('./embeddings/2_test_ready_to_ml.csv', index=False)

# 여기

In [13]:
train_to_ml = pd.read_csv('./embeddings/1_train_ready_to_ml.csv')
test_ready_to_ml = pd.read_csv('./embeddings/2_test_ready_to_ml.csv')

In [16]:
train_to_ml.head(5)

,first_party_berted,second_party_berted,facts_berted,first_party_winner
0,"tensor([[ 5.0364e-03, 6.4395e-03, -1.2888e-02...","tensor([[-2.3282e-02, 8.4174e-03, 1.4726e-02...","tensor([[-4.9459e-02, 2.7671e-03, 3.6642e-02...",1
1,"tensor([[-3.9016e-03, 2.6781e-02, -4.4628e-02...","tensor([[ 2.9523e-03, 1.2450e-02, -5.3252e-02...","tensor([[-4.4846e-02, -1.8939e-03, 1.4574e-02...",0
2,"tensor([[ 3.0499e-03, 9.2587e-03, -1.7064e-02...","tensor([[ 9.8794e-03, -1.9934e-02, -5.2907e-02...","tensor([[-4.7123e-02, -6.5828e-03, 1.5028e-02...",1
3,"tensor([[-1.5234e-02, -8.7536e-04, 1.5897e-02...","tensor([[ 1.5804e-02, 2.7409e-04, -5.3871e-02...","tensor([[-8.0872e-02, -2.1650e-02, 9.3573e-03...",0
4,"tensor([[-1.1956e-02, -9.9873e-03, 2.0198e-02...","tensor([[-7.6583e-03, 6.1737e-03, -4.3932e-02...","tensor([[-4.2441e-02, -1.2599e-02, 3.4638e-02...",1


In [15]:
test_ready_to_ml.head(5)

,first_party_berted,second_party_berted,facts_berted
0,"tensor([[-6.6722e-03, -8.0011e-02, -1.4580e-02...","tensor([[ 2.6328e-02, -1.6656e-02, -2.0892e-02...","tensor([[-4.1743e-02, -4.1057e-02, 3.7544e-02..."
1,"tensor([[-3.0531e-03, -1.1769e-02, -3.4803e-02...","tensor([[ 1.0108e-02, 1.5938e-03, 6.1594e-03...","tensor([[-6.2039e-02, -9.7709e-03, 3.8074e-02..."
2,"tensor([[-2.5638e-02, -1.6005e-02, -2.1478e-02...","tensor([[ 1.4963e-02, 1.2814e-02, -2.9757e-02...","tensor([[-5.1186e-02, -2.1976e-05, 2.2741e-02..."
3,"tensor([[ 2.8991e-02, 3.6599e-02, -5.9128e-02...","tensor([[ 2.6328e-02, -1.6656e-02, -2.0892e-02...","tensor([[-5.2218e-02, -2.7130e-02, 2.2865e-02..."
4,"tensor([[ 3.6941e-02, -3.9247e-02, -2.6675e-02...","tensor([[ 3.2124e-02, -1.3590e-02, -4.0433e-02...","tensor([[-3.7322e-02, 4.6000e-03, 2.6736e-02..."


train_to_ml_fp_pr = dlc.tensor_separator(train_to_ml, 'first_party_berted')
train_to_ml_sp_pr = dlc.tensor_separator(train_to_ml, 'second_party_berted')
train_to_ml_facts_pr = dlc.tensor_separator(train_to_ml, 'facts_berted')

train_to_ml_fp_pr = train_to_ml_fp_pr.astype('float64')
train_to_ml_sp_pr = train_to_ml_sp_pr.astype('float64') 
train_to_ml_facts_pr = train_to_ml_facts_pr.astype('float64')


train_to_ml_fp_pr.info()

test_to_ml_fp_pr = dlc.tensor_separator(test_ready_to_ml, 'first_party_berted')
test_to_ml_sp_pr = dlc.tensor_separator(test_ready_to_ml, 'second_party_berted')
test_to_ml_facts_pr = dlc.tensor_separator(test_ready_to_ml, 'facts_berted')

test_to_ml_fp_pr = test_to_ml_fp_pr.astype('float64')
test_to_ml_sp_pr = test_to_ml_sp_pr.astype('float64')
test_to_ml_facts_pr = test_to_ml_facts_pr.astype('float64')

to_be_X = pd.concat([train_to_ml_fp_pr, train_to_ml_sp_pr, train_to_ml_facts_pr], axis=1)
to_be_test_x = pd.concat([test_to_ml_fp_pr, test_to_ml_sp_pr, test_to_ml_facts_pr], axis=1)


to_be_X.columns = np.arange(len(to_be_X.columns))
to_be_X = pd.concat([to_be_X, train_to_ml['first_party_winner']], axis =1)
to_be_X

to_be_test_x.columns = np.arange(len(to_be_test_x.columns))
to_be_test_x

In [17]:
!mkdir -p 'model'
predictor = TabularPredictor(label='first_party_winner',path='model').fit(train_to_ml, presets=['best_quality'])


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "model/"
AutoGluon Version:  0.8.0
Python Version:     3.10.9
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.5.0: Mon Apr 24 20:53:44 PDT 2023; root:xnu-8796.121.2~5/RELEASE_ARM64_T8103
Disk Space Avail:   34.88 GB / 245.11 GB (14.2%)
Train Data Rows:    2478
Train Data Columns: 3
Label Column: first_party_winner
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the proble

	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 405.2s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("model/")


In [18]:
preds_solid = predictor.predict(test_ready_to_ml)
preds_proba = predictor.predict_proba(test_ready_to_ml)


In [24]:
print(preds_solid.value_counts())
preds_proba = preds_proba.rename(columns={0: 'p0', 1:'p1'})


1    1238
0       2
Name: first_party_winner, dtype: int64


In [34]:
temp = pd.DataFrame({'temp' : [i for i in range(len(preds_proba))]})
preds_proba_temp = pd.concat([preds_proba, temp], axis=1)
preds_proba_temp

,p0,p1,temp
0,0.401634,0.598366,0
1,0.407809,0.592191,1
2,0.224601,0.775399,2
3,0.344396,0.655604,3
4,0.290013,0.709987,4
...,...,...,...
1235,0.291976,0.708024,1235
1236,0.420261,0.579739,1236
1237,0.340529,0.659471,1237
1238,0.374998,0.625002,1238


In [41]:
for p0_value_index in range(len(preds_proba_temp['p0'])):
#     print(p0_value_index)
#     print(preds_proba_temp['p0'][p0_value_index])
    if preds_proba_temp['p0'][p0_value_index] < 0.4:
        preds_proba_temp['temp'][p0_value_index] = 0
    else:
        preds_proba_temp['temp'][p0_value_index] = 1
        
preds_proba_temp.rename_col
        
    

,p0,p1,temp
0,0.401634,0.598366,1
1,0.407809,0.592191,1
2,0.224601,0.775399,0
3,0.344396,0.655604,0
4,0.290013,0.709987,0
...,...,...,...
1235,0.291976,0.708024,0
1236,0.420261,0.579739,1
1237,0.340529,0.659471,0
1238,0.374998,0.625002,0


In [21]:
final_submission = pd.concat([sample_submission['ID'], preds_solid], axis=1)
final_submission

,ID,first_party_winner
0,TEST_0000,1
1,TEST_0001,1
2,TEST_0002,1
3,TEST_0003,1
4,TEST_0004,1
...,...,...
1235,TEST_1235,1
1236,TEST_1236,1
1237,TEST_1237,1
1238,TEST_1238,1


In [22]:
final_submission.to_csv("./bert_autogluon_submission_0630.csv", index=False)